In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

* Below function will preprocess the data, split the data in train and test, and finally train an Ensemble Random Forest

In [6]:
def preprocess(df):

    df_labels = pd.read_csv('train_labels.csv')
    
    missing_values = df.isnull().sum()
    percentage_missing = (missing_values / df.shape[0]) * 100
    columns_to_keep = percentage_missing[percentage_missing < 50].index
    
    df = df[columns_to_keep]
    
    df = pd.merge(df,df_labels,on='customer_ID')
    df = df.drop(columns=['customer_ID','S_2'])

    
    int_mean = df.select_dtypes(include=['int64']).mean()
    float_median = df.select_dtypes(include=['float64']).median()
    mode = df.select_dtypes(include=['object']).mode().iloc[0]

    
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = df[col].fillna(int_mean[col])
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].fillna(float_median[col])
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].fillna(mode[col])


    df['D_63'] = df['D_63'].map(df['D_63'].value_counts(normalize=True))
    df['D_64'] = df['D_64'].map(df['D_64'].value_counts(normalize=True))

    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['target']),df['target'],test_size=0.2,
                                                        random_state=42)

    model = RandomForestClassifier(n_estimators=10, warm_start=True, n_jobs=-1)

    # Train the model with the initial number of trees
    model.fit(X_train, y_train)
    
    # Increase the number of trees and continue training
    model.n_estimators += 5
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    i = 1
    if i==550:
        print(classification_report(y_test,y_pred))
    else:
        i = i+1

In [7]:
import time
start_time = time.time()
for chunk in pd.read_csv('train_data.csv', chunksize=100000):
    # Preprocess the chunk
    preprocess(chunk)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 779.6397974491119 seconds


In [ ]:
model